In [1]:
!aws sagemaker update-user-profile --domain-id d-ugpjmtq9damp --user-profile-name uche\
    --user-settings JupyterLabAppSettings={CodeRepositories=[{RepositoryUrl=https://github.com/aws-samples/sagemaker-ssh-helper.git}]}


An error occurred (ResourceNotFound) when calling the UpdateUserProfile operation: UserProfile [arn:aws:sagemaker:us-east-1:715253196401:user-profile/d-ugpjmtq9damp/uche] does not exist.


In [4]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker', region_name = 'us-east-2')

# Parameters
domain_id = 'd-ugpjmtq9damp'
user_profile_name = 'uche'
repository_url = 'https://github.com/Kaggle/kaggle-api.git'

# Update the user profile
# response = sagemaker_client.update_user_profile(
#     DomainId=domain_id,
#     UserProfileName=user_profile_name,
#     UserSettings={
#         'JupyterLabAppSettings': {
#             'CodeRepositories': [
#                 {
#                     'RepositoryUrl': repository_url
#                 }
#             ]
#         }
#     }
# )

# # Print the response (optional)
# print(response)



import boto3

# Initialize the SageMaker client
# sagemaker_client = boto3.client('sagemaker', region_name='your-region')

# Parameters
domain_id = 'd-ugpjmtq9damp'
user_profile_name = 'uche'

# Update the domain
response = sagemaker_client.update_domain(
    DomainId=domain_id,
    DefaultUserSettings={
        'JupyterLabAppSettings': {
            'CodeRepositories': [
                {
                    'RepositoryUrl': repository_url
                }
            ]
        }
    }
)

# Print the response (optional)
print(response)

{'DomainArn': 'arn:aws:sagemaker:us-east-2:715253196401:domain/d-ugpjmtq9damp', 'ResponseMetadata': {'RequestId': '8f091816-39b6-4b06-bbbd-966667fb4863', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8f091816-39b6-4b06-bbbd-966667fb4863', 'content-type': 'application/x-amz-json-1.1', 'content-length': '78', 'date': 'Thu, 20 Mar 2025 23:32:20 GMT'}, 'RetryAttempts': 0}}


In [1]:
import os
import boto3
import re
import sagemaker
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name
session = Session()
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sklearn import preprocessing
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-dist-algo"

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
import os
import boto3
import re
import sagemaker
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name
session = Session()

output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "algo-dist-xgb")
hyperparams = {
    "objective": "binary:logistic",
    "num_round": "500",
    "verbosity": "3",
    "tree_method": "gpu_hist",
    "eval_metric": "auc",
    "use_dask_gpu_training": "true"
}

content_type = "application/x-parquet"
instance_type = "ml.g5.12xlarge"

xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
xgb_script_mode_estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_container,
    hyperparameters=hyperparams,
    role=role,
    instance_count=1,
    instance_type=instance_type,
    output_path=output_path,
    max_run=7200,
    # volume_size=200,
    keep_alive_period_in_seconds = 1000,

)

train_input = TrainingInput(
    "s3://sagemaker-us-east-1-715253196401/sagemaker/DEMO-xgboost-dist-algo/train_xgb_micro_pq/", 
    content_type=content_type,
    # distribution = 'ShardedByS3Key'
)

validation_input = TrainingInput(
    "s3://sagemaker-us-east-1-715253196401/sagemaker/DEMO-xgboost-dist-algo/test_xgb_large_pq/", 
    content_type=content_type,
       # distribution = 'ShardedByS3Key'
)

xgb_script_mode_estimator.fit({"train": train_input})
                               # , "validation": validation_input})

[03/11/25 21:17:17] INFO     Ignoring unnecessary instance type: None.                            ]8;id=13801;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=895745;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=712324;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=772292;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=672776;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=166967;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-03-11-21-17-17-919                                             

2025-03-11 21:17:22 Starting - Found matching resource for reuse
2025-03-11 21:17:22 Downloading - Downloading input data...
2025-03-11 21:17:38 Training - Training image download completed. Training in progress.[2025-03-11 21:17:41.677 ip-10-2-113-219.ec2.internal:13 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-03-11 21:17:41.747 ip-10-2-113-219.ec2.internal:13 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-03-11:21:17:42:INFO] Imported framework sagemaker_xgboost_container.training
[2025-03-11:21:17:42:INFO] Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
[2025-03-11:21:17:42:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2025-03-11:21:17:42:INFO] Failed to parse hyperparameter tree_method value gpu_hist to Json.
Returning the value itself
[2025-03-11:21:17:42:INFO] Running XGBoost Sagemaker in algorithm mode
[2025-03-11:21:17:42:INFO] Determined 4

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:52                                                                                   │
│                                                                                                  │
│   49 │      # distribution = 'ShardedByS3Key'                                                    │
│   50 )                                                                                           │
│   51                                                                                             │
│ ❱ 52 xgb_script_mode_estimator.fit({"train": train_input})                                       │
│   53 │   │   │   │   │   │   │      # , "validation": validation_input})                         │
│   54                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:167 in wrapper  │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:138 in wrapper  │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                            